In [118]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

In [119]:
df=pd.read_csv('train.csv')
df_test=pd.read_csv('test.csv')
df_subb=pd.read_csv('sample_submission.csv')

In [120]:
def createfeatures(df,file_name,scaler=None):
    df.loc[df['last_new_job']=='never','last_new_job']=0
    df.loc[df['last_new_job']=='>4','last_new_job']=5
    imp = SimpleImputer(missing_values=np.nan, strategy='mean')
    imp.fit(df['last_new_job'].values.reshape(-1, 1))
    values=imp.transform(df['last_new_job'].values.reshape(-1, 1))
    df['last_new_job']=np.round_(values)
    df.loc[df['experience']=='>20','experience']=25
    df.loc[df['experience']=='<1','experience']=0.5
    imp = SimpleImputer(missing_values=np.nan, strategy='mean')
    imp.fit(df['experience'].values.reshape(-1, 1))
    values=imp.transform(df['experience'].values.reshape(-1, 1))
    df['experience']=np.round_(values)
    df=df.fillna('empty')
    df = pd.get_dummies(df, columns = ['city','gender','relevent_experience','enrolled_university','education_level','major_discipline','company_size','company_type'])
    if scaler is None:
        scaler = StandardScaler()
        scaler.fit(df)
    df=pd.DataFrame(scaler.transform(df),columns = df.columns)
    df.to_csv(file_name,index=False)
    return df,scaler

In [121]:
Y=df[['target','enrollee_id']]
df.drop(['target','enrollee_id'],inplace=True,axis=1)
Y.to_csv('train_y.csv',index=False)
df,scaler=createfeatures(df,'train_x.csv')

In [125]:
y_test=df_test['enrollee_id']
y_test.to_csv('test_y.csv',index=False)
df_test.drop(['enrollee_id'],axis=1,inplace=True)
df_test,_=createfeatures(df_test,'test_x.csv',scaler)